<a href="https://colab.research.google.com/github/aml-spring-19/homework-5-cv5/blob/master/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

# plots
import matplotlib.pyplot
import seaborn as sns
%matplotlib inline

# processing & model selection
from keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV, train_test_split

# models
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.wrappers.scikit_learn import KerasClassifier


np.random.seed(42)

Using TensorFlow backend.


# Homework 5

### Task 1

Task is to run a multilayer perceptron with two hidden layers and relu activations, using the Keras Sequential Interface. We also tune for regularization strength and number of hidden units. 

Importing the data:

In [2]:
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Map the target variable:

In [3]:
print(df.species.unique())

['setosa' 'versicolor' 'virginica']


In [0]:
mapping = {'setosa': 0, 'versicolor': 1, 'virginica': 2}
df.species.replace(mapping, inplace = True)

Split our dataset into training and test sets. 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('species', axis=1), df.species, test_size=0.2)

Encode the target:

In [0]:
num_species = len(df['species'].unique())

y_train = to_categorical(y_train, num_species)
y_test = to_categorical(y_test, num_species)

Make our model:

In [7]:
X_train.shape

(120, 4)

In [0]:
def make_model(hidden_size, reg_strength, optimizer='adam'):
    """
    Create the model to pass to the Keras Classifier
    """
    
    model = Sequential([
        Dense(hidden_size, input_shape=(4,), activation='relu',   # first layer
              kernel_regularizer=regularizers.l2(reg_strength)),
        Dense(hidden_size, activation='relu',                     # second layer
              kernel_regularizer=regularizers.l2(reg_strength)),
        Dense(3, activation='softmax')                            # output layer
    ])
    
    # compile the above model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [19]:
clf = KerasClassifier(make_model, epochs=10, verbose=0)

param_grid = {'hidden_size': [32, 64, 128],
              'reg_strength': np.logspace(-3, -1, 5)}

grid = GridSearchCV(clf, param_grid=param_grid, cv=3)
grid.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f97d867a550>,
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'hidden_size': [16, 32, 64], 'reg_strength': array([0.001  , 0.00316, 0.01   , 0.03162, 0.1    ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
print('Best score: {}'.format(np.round(grid.best_score_, 2)))
print('Best params: {}'.format(grid.best_params_))

Best score: 0.89
Best params: {'hidden_size': 64, 'reg_strength': 0.001}


In [25]:
score = grid.score(X_test, y_test)
print('Test score: {}'.format(np.round(score, 2)))

Test score: 0.9


### Task 2

Train a multilayer perceptron (fully connected) on the Fashion MNIST dataset. 
1. Use 10000 samples from the training set for model selection and to compute learning curves (accuracy vs epochs, not vs n_samples).
2. Compare the following models and plot their learning curves:
    - Vanilla Model
    - Model using dropout
    - Model using batch normalization and residual connections (but no dropout)





In [20]:
# loading the data
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

NameError: ignored